In [1]:
import sys
import tensorflow as tf
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)
SEED=27

tf.random.set_seed(SEED)

print('Python version:', sys.version)
print('Tensorflow version:', tf.__version__)

Python version: 3.8.9 (default, Apr  3 2021, 01:50:09) 
[Clang 12.0.0 (clang-1200.0.32.29)]
Tensorflow version: 2.4.1


## Загрузка сырых данных

In [2]:
def get_3d_quartile_words_count(sentences_array):
    """
    Получает массив строк/предложений.
    Отдает число N.
    75% всех предложений данного массива не превышают N слов в длину.
    """
    def count_words(sentence):
        return len(sentence.split(' '))
        
    
    return round(
        np.percentile(sentences_array.apply(count_words), 75)
    )

orig_toxic_comments_df = pd.read_csv('data/toxicCommentsOriginalDF.csv')
print('Количество комментариев:', len(orig_toxic_comments_df))
print(f"75% всех комментариев не больше {get_3d_quartile_words_count(orig_toxic_comments_df['comment'])} слов в длину")
orig_toxic_comments_df.head(3)

Количество комментариев: 14412
75% всех комментариев не больше 32 слов в длину


,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n",1.0
2,Собаке - собачья смерть\n,1.0


## Split train-validation-test DF

In [3]:
df = orig_toxic_comments_df.copy()

y = df.pop('toxic')
X = np.array(df)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

## Train model

In [5]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers, losses

# local util
from textPreprocessing import preprocess_text

In [6]:
MAX_FEATURES = 10000
MAX_SEQUENCE_LENGTH = get_3d_quartile_words_count(df['comment'])


def decode_byte_string_to_russian_text(string_tensor):
    try:
        return string_tensor[0].numpy().decode('utf-8')
    except:
        return ''

def custom_standardization(texts_tensor):    
    result = tf.map_fn(fn=lambda x: preprocess_text(decode_byte_string_to_russian_text(x)), elems=texts_tensor)
    
    return result

vectorize_layer = TextVectorization(
#     standardize=custom_standardization,
    max_tokens=MAX_FEATURES,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH
)
vectorize_layer.adapt(X_train)

In [7]:
model = tf.keras.Sequential([
    layers.Input(shape=[], dtype=tf.string),
    vectorize_layer,
    layers.Embedding(
        input_dim = len(vectorize_layer.get_vocabulary()),
        output_dim = 16
    ),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(1),
    layers.Activation('sigmoid') # то есть в конце мы выдаем уже вероятности
])

model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]
)

In [8]:
epochs = 20
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=epochs,
    validation_split=0.2, # сколько от тестовой выборки отрезать под валидационную
)

Epoch 1/20
289/289 [==============================] - 2s 5ms/step - loss: 0.6658 - binary_accuracy: 0.6402 - recall: 0.0908 - precision: 0.4247 - val_loss: 0.6085 - val_binary_accuracy: 0.6670 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/20
289/289 [==============================] - 1s 2ms/step - loss: 0.5998 - binary_accuracy: 0.6620 - recall: 0.0090 - precision: 0.5528 - val_loss: 0.5673 - val_binary_accuracy: 0.6891 - val_recall: 0.0729 - val_precision: 0.9180
Epoch 3/20
289/289 [==============================] - 1s 2ms/step - loss: 0.5487 - binary_accuracy: 0.7062 - recall: 0.1410 - precision: 0.8869 - val_loss: 0.5263 - val_binary_accuracy: 0.7493 - val_recall: 0.2956 - val_precision: 0.8598
Epoch 4/20
289/289 [==============================] - 1s 2ms/step - loss: 0.5023 - binary_accuracy: 0.7705 - recall: 0.4005 - precision: 0.8534 - val_loss: 0.4898 - val_binary_accuracy: 0.7944 - val_recall: 0.4948 - val_precision: 0.8155
Epoch 5/20
289/289 [====================

In [9]:
loss, accuracy, recall, precision = model.evaluate(x=X_test, y=y_test)

print("Loss: ", loss)
print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("Precision: ", precision)

91/91 [==============================] - 0s 821us/step - loss: 0.3513 - binary_accuracy: 0.8647 - recall: 0.7872 - precision: 0.8006
Loss:  0.3513270914554596
Accuracy:  0.8647242188453674
Recall:  0.7872117161750793
Precision:  0.8006396293640137


In [10]:
tf.math.confusion_matrix(labels=y_test, predictions=(model.predict(X_test) > 0.5) * 1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1742,  187],
       [ 203,  751]], dtype=int32)>